In [1]:
import sys
sys.path.append('../')
import pandas as pd
from bunkatopics import Bunka
import random
from sklearn.datasets import fetch_20newsgroups

full_docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
full_docs = random.sample(full_docs, 5000)

#### Instantiate BunkaTopics

#### Add the Embedding Model you wish from Langchain

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
#embedding_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

bunka = Bunka(model_hf=embedding_model)

bunka.fit(full_docs)

INFO      |2023-06-07 15:00:04|{}|fit|Extracting Terms
100%|█████████████████████████████████████████████████████████████████████████| 5000/5000 [03:20<00:00, 24.93it/s]
INFO      |2023-06-07 15:03:27|{}|fit|Embedding Documents, this may take few minutes
INFO      |2023-06-07 15:04:33|{}|fit|Reducing Dimensions
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


#### Get the topics

In [3]:
df_topics = bunka.get_topics(n_clusters = 5)

In [4]:
topic_fig = bunka.visualize_topics(width=800, height=800)

### Get the Bourdieu Visualization

In [5]:

bourdieu_fig = bunka.visualize_bourdieu(x_left_words=["past"],
                                        x_right_words=["future", "futuristis"],
                                        y_top_words=["politics", "Government"],
                                        y_bottom_words=["cultural phenomenons"],
                                        height=2000,
                                        width=2000, 
                                        clustering =False)


#### Get the most speficic documents per topic

In [6]:
top_doc = bunka.get_top_documents(top_docs=5)

#### Get the Distribution of topics

In [7]:
fig_repartition = bunka.get_topic_repartition(width=1200, height=800)

#### Search your data using Vector Search

In [8]:
df_search = bunka.search('Space Exploration')

#### Compute Topic Coherence

In [9]:
topic_coherence = bunka.get_topic_coherence()
topic_coherence

0.6518552907085245

In [10]:
### Add GPT Embeddings